# Загружаем исходные данные к сегодняшнему семинару ✈

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv
Saving dataset_ad.csv to dataset_ad.csv
Saving dataset_cost.csv to dataset_cost.csv


In [ ]:
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['dataset (2).csv']))
df_spend = pd.read_csv(io.BytesIO(uploaded['dataset_cost (2).csv']))
df_ads = pd.read_csv(io.BytesIO(uploaded['dataset_ad (2).csv']))

display(df)
display(df_spend)
display(df_ads)

KeyError: 'dataset (1).csv'

# Сравнение метрик органики и закупки по когортам ⛲

In [ ]:
df['event_date'] = pd.to_datetime(df['event_date'])
df['install_date'] = pd.to_datetime(df['install_date'])

df['start_of_week'] = df['install_date'] - pd.to_timedelta(df['install_date'].dt.dayofweek, unit='d')

In [ ]:
df_compare = df[df['install_date'] >= '2022-04-01'].groupby(['start_of_week', 'is_organic'], as_index = False).agg({'appsflyer_id':'nunique', 'revenue_usd':'sum', 'ad_revenue_usd':'sum', 'payments':'sum'}).rename(columns = {'appsflyer_id':'installs'})

df_compare

Дополним полученные результаты метриками общей выручки и LTV (RPI), а также посчитаем net LTV, с условием channel cost: для iap выручки channel cost = 15%, для рекламной монетизации = 5%.

In [ ]:
df_compare['total_revenue_gross'] = df_compare['revenue_usd'] + df_compare['ad_revenue_usd']
df_compare['ltv_gross'] = df_compare['total_revenue_gross'] / df_compare['installs']
df_compare['payments_per_user'] = df_compare['payments'] / df_compare['installs']

df_compare['total_revenue_net'] = df_compare['revenue_usd']*0.85 + df_compare['ad_revenue_usd']*0.95
df_compare['ltv_net'] = df_compare['total_revenue_net'] / df_compare['installs']

df_compare

Посмотрим также какой вклад в общую выручку дают внутренние покупки и рекламная монетизация.

In [ ]:
df_compare['iap_share'] = df_compare['revenue_usd'] / df_compare['total_revenue_gross']
df_compare['ads_share'] = df_compare['ad_revenue_usd'] / df_compare['total_revenue_gross']

df_compare

Построим графики, чтобы сравнить метрики органики и трафика между собой. Какие выводы можно сделать из полученных результатов?

In [ ]:
import plotly.express as px

fig_installs = px.line(df_compare, x = 'start_of_week', y = 'installs', color = 'is_organic')
fig_installs.show()

In [ ]:
fig_ltv_gross = px.bar(df_compare, x = 'start_of_week', y = 'ltv_gross', color = 'is_organic', barmode = 'group')
fig_ltv_gross.show()

In [ ]:
fig_ltv_net = px.bar(df_compare, x = 'start_of_week', y = 'ltv_net', color = 'is_organic', barmode = 'group')
fig_ltv_net.show()

In [ ]:
fig_iap_share = px.line(df_compare, x = 'start_of_week', y = 'iap_share', color = 'is_organic')
fig_iap_share.show()

In [ ]:
fig_ads_share = px.line(df_compare, x = 'start_of_week', y = 'ads_share', color = 'is_organic')
fig_ads_share.show()

Какие выводы можно сделать при сравнении монетизационных метрик когорт органики и закупки?

Какие еще метрики было бы интересно сравнить для когорт органики и закупки на ваш взгляд? 🤷

Посмотрим какая кампания перформила лучше всего, для этого нам нужно знать маркетинговые затраты по каждой из кампаний.

In [ ]:
df_users = df[(df['is_organic'] == False) & (df['install_date'] >= '2022-04-01')].groupby(['appsflyer_id', 'campaign_name'], as_index = False).agg({'revenue_usd':'sum', 'ad_revenue_usd':'sum'})
df_users

In [ ]:
df_campaigns = pd.merge(df_users, df_spend, on='appsflyer_id')
df_campaigns

In [ ]:
df_campaigns = df_campaigns.groupby('campaign_name').agg({'appsflyer_id':'nunique', 'user_cost':'sum', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'installs'})
df_campaigns

Рассчитаем маржинальность кампаний через ROAS, в котором учитывается net LTV.

In [ ]:
df_campaigns['roas'] = (df_campaigns['revenue_usd']*0.85 + df_campaigns['ad_revenue_usd']*0.95)*100/df_campaigns['user_cost']

df_campaigns = df_campaigns.sort_values(by = 'roas', ascending = False)
df_campaigns

Что можно сказать о полученных результатах?

# Инкрементальность ⛅

Из полученных результатов выше мы видим, что в данных есть некоторые погрешности с атрибуцированием пользователей и часть инсталлов, которые промаркированы в данных как "органика" имеют маркетинговый кост.

Возможны два основных варианта: один и тот же пользователь в части данных промаркирован как органический, но в то же время в некоторых записях указан как трафик и таким образом имеем дубли по пользователям.

Второй вариант - пользователь в агрегированных дневных данных промаркирован как органический, а в данных по костам имеет спенд, тогда имеем ошибку в атрибуцировании пользователей в агрегированных данных.

Проверим, есть ли в агрегированных данных пользователи, у которых есть две и более кампаний.

In [ ]:
df_check = df.groupby('appsflyer_id', as_index = False).agg({'campaign_name':'nunique'}).rename(columns = {'campaign_name':'campaigns'})
df_check = df_check.sort_values(by = 'campaigns', ascending = False)

df_check

Очевидно в данных присутствуют дубли по атрибуцированию пользователей, посмотрим их долю.

In [ ]:
df_check = df_check.groupby('campaigns', as_index = False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'users'})
df_check

Какие выводы можно сделать из полученных результатов? Каким образом стоит работать с инсталлами в данном случае, на ваш взгляд?

# Сравнение метрик рекламных сеток

Давайте посмотрим какие сетки крутят рекламу для пользователей из США и сравним их метрики 🌍

In [ ]:
df_ads = df_ads[df_ads['country_code'] == 'US'].groupby('ad_network', as_index=False).agg({'impressions':'sum', 'revenue_usd':'sum'})
df_ads['ecpm'] = df_ads['revenue_usd']*1000/df_ads['impressions']

df_ads

Какая из сеток перформит лучше всего? ♥

# **Домашнее задание:**

Какие выводы можно сделать при сравнении монетизационных метрик когорт органики и закупки?

Посчитайте дополнительные показатели, если необходимо

И напишите выводы по вопросам из воркбука семинара

Посмотрим внутренние покупки и рекламную монетизацию по каждой из кампаний.

In [ ]:
df_compare_camp = df_users.groupby(['campaign_name'], as_index = False).agg({'appsflyer_id':'nunique', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'installs'})

df_compare_camp

Для того, чтобы получить чистые данные при расчете метрик привяжем к каждому пользователю одну версию - ту, с которой он зашел в игру

In [ ]:
df_start_ver = df[df['event_date'] == df['install_date']][['appsflyer_id', 'app_version']].rename(columns = {'app_version':'start_version'})
df = pd.merge(df, df_start_ver, on='appsflyer_id')
df

Так как мы будем рассчитывать в том числе retention rate и LTV по каждой из версий, нам необходимо ограничить датасет временными рамками по когортам срок жизни которых достиг 30 дней и более. Напомним, что максимальный event date = 2022-07-31

In [ ]:
df_versions = df[(df['install_date'] >= '2022-04-01') & (df['install_date'] <= '2022-07-01')][['event_date', 'start_version', 'install_date', 'appsflyer_id', 'is_organic', 'revenue_usd', 'ad_revenue_usd', 'sessions', 'playtime']]
df_versions['activity_day'] = (df_versions['event_date']-df_versions['install_date']).dt.days
df_versions

Посмотрим инсталлы по каждой версии и оставим для расчета топ-3 по инсталлам

In [ ]:
df_top = df_versions.groupby('start_version', as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'users'}).sort_values(by = 'users', ascending=False)
top = list(df_top.head(3)['start_version'].values)
top

In [ ]:
df_versions = df_versions[df_versions['start_version'].isin(top)]
df_versions

Рассчитаем retention rate для каждого билда за 0-30 день активности

In [ ]:
df_ret = df_versions[(df_versions['activity_day']>=0) & (df_versions['activity_day']<=30)].groupby(['start_version', 'activity_day'], as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'users'})
df_ret

In [ ]:
df_installs = df_ret.groupby('start_version', as_index=False).agg({'users':'max'}).rename(columns = {'users':'installs'})

df_ret = pd.merge(df_ret, df_installs, on='start_version')
df_ret['ret_rate'] = df_ret['users'] / df_ret['installs']

df_ret

In [ ]:
fig1 = px.line(df_ret, x='activity_day', y='ret_rate', color='start_version')
fig1.show()

Для лучшего понимания деталей посмотрим на retention rate по версиям для 1-10 дней активности

In [ ]:
fig2 = px.line(df_ret, x='activity_day', y='ret_rate', color='start_version', range_x=[1,10], range_y=[0,0.3])
fig2.show()

Посмотрим средние значения метрик вовлеченности и LTV по топ-3 версиям

In [ ]:
df_eng = df_versions.groupby(['appsflyer_id', 'start_version'], as_index=False).agg({'sessions':'sum', 'playtime':'sum', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'})
df_eng = df_eng.groupby('start_version', as_index = False).agg({'appsflyer_id':'nunique', 'sessions':'sum', 'playtime':'sum', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'users'})

df_eng

In [ ]:
df_eng['sessions_per_user'] = df_eng['sessions'] / df_eng['users']
df_eng['playtime_per_user'] = df_eng['playtime'] / df_eng['users']
df_eng['average_session_length'] = df_eng['playtime'] / df_eng['sessions']
df_eng['ltv'] = (df_eng['revenue_usd']+df_eng['ad_revenue_usd']) / df_eng['users']

df_eng

Метрики роста не подходят для построения аналитики продукта, так как на них влияет не только продукт, но и маркетинг, и продвижение;
Две ключевые метрики продукта — LTV и CAC;
LTV — высокоуровневая метрика, поэтому ее следует декомпозировать на метрики продукта, привязанные к ключевым этапам жизненного цикла пользователя в приложении;
Суть когортного анализа состоит в том, чтобы отслеживать ключевые метрики каждой отдельной когорты во времени;
Когортный анализ позволяет объективно сравнивать разные версии продукта и оценивать влияние изменений на продукт.